In [26]:
import http.client
import json
import pandas as pd
from tqdm import tqdm

In [28]:
import http.client
import json
import pandas as pd
from tqdm import tqdm

# Define all states with coordinates
states = {
    # Northern
    'Delhi': (28.6139, 77.2090),
    'Haryana': (29.0588, 76.0856),
    'Himachal Pradesh': (32.0657, 77.3444),
    'Jammu and Kashmir': (33.7782, 76.5762),
    'Punjab': (31.1471, 75.3412),
    'Rajasthan': (27.0238, 74.2179),
    'Uttar Pradesh': (26.8467, 80.9462),
    'Uttarakhand': (30.0668, 79.0193),
    'Chandigarh': (30.7333, 76.7794),

    # Western
    'Goa': (15.2993, 74.1240),
    'Gujarat': (22.2587, 71.1924),
    'Madhya Pradesh': (23.2599, 77.4126),
    'Chhattisgarh': (21.2787, 81.6550),
    'Maharashtra': (19.7515, 75.7139),
    'Dadra & Nagar Haveli and Daman & Diu': (20.1807, 73.0169),

    # Southern
    'Andhra Pradesh': (15.9129, 79.7400),
    'Telangana': (17.1231, 79.2088),
    'Karnataka': (15.3173, 75.7139),
    'Kerala': (10.8505, 76.2711),
    'Tamil Nadu': (11.1271, 78.6569),
    'Puducherry': (11.9416, 79.8083),

    # Eastern
    'Bihar': (25.0961, 85.3131),
    'West Bengal': (22.9868, 87.8547),
    'Jharkhand': (23.6100, 85.2799),
    'Odisha': (20.9517, 85.0985),
    'Sikkim': (27.5330, 88.6130),

    # North Eastern
    'Assam': (26.2006, 92.9376),
    'Arunachal Pradesh': (27.0841, 93.6053),
    'Meghalaya': (25.4670, 91.3662),
    'Tripura': (23.9395, 91.9882),
    'Manipur': (24.6637, 93.9063),
    'Nagaland': (26.1584, 94.5624),
    'Mizoram': (23.1645, 92.9376)
}

# Map state to region
state_to_region = {
    'Delhi': 'Northern', 'Haryana': 'Northern', 'Himachal Pradesh': 'Northern',
    'Jammu and Kashmir': 'Northern', 'Punjab': 'Northern', 'Rajasthan': 'Northern',
    'Uttar Pradesh': 'Northern', 'Uttarakhand': 'Northern', 'Chandigarh': 'Northern',
    'Goa': 'Western', 'Gujarat': 'Western', 'Madhya Pradesh': 'Western',
    'Chhattisgarh': 'Western', 'Maharashtra': 'Western',
    'Dadra & Nagar Haveli and Daman & Diu': 'Western',
    'Andhra Pradesh': 'Southern', 'Telangana': 'Southern', 'Karnataka': 'Southern',
    'Kerala': 'Southern', 'Tamil Nadu': 'Southern', 'Puducherry': 'Southern',
    'Bihar': 'Eastern', 'West Bengal': 'Eastern', 'Jharkhand': 'Eastern',
    'Odisha': 'Eastern', 'Sikkim': 'Eastern',
    'Assam': 'North Eastern', 'Arunachal Pradesh': 'North Eastern',
    'Meghalaya': 'North Eastern', 'Tripura': 'North Eastern',
    'Manipur': 'North Eastern', 'Nagaland': 'North Eastern',
    'Mizoram': 'North Eastern'
}

# API setup
conn = http.client.HTTPSConnection("meteostat.p.rapidapi.com")
headers = {
    'x-rapidapi-key': "596374651amsha3f93aa550a1ed8p197f17jsn172c1ab92db4",  # Replace with your actual API key
    'x-rapidapi-host': "meteostat.p.rapidapi.com"
}

# Dates
start_date = "2023-01-01"
end_date = "2025-03-31"

# Collect all data
all_data = []

for state_name, (lat, lon) in tqdm(states.items(), desc="Fetching weather data"):
    conn.request("GET", f"/point/monthly?lat={lat}&lon={lon}&alt=200&start={start_date}&end={end_date}", headers=headers)
    res = conn.getresponse()
    raw = res.read().decode("utf-8")
    json_data = json.loads(raw)

    if 'data' in json_data:
        df = pd.DataFrame(json_data['data'])
        print(f"Columns for {state_name}: {df.columns}")  # Check columns in data
        if 'month' in df.columns:
            df['date'] = pd.to_datetime(df['month']).dt.date  # Keep only date
            df.drop(columns=['month'], inplace=True)
        else:
            print(f"⚠️ No 'month' column for {state_name}, checking available columns.")
            # If no 'month' column, check what columns are available
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date']).dt.date  # Adjust according to actual date column name
            else:
                print(f"⚠️ No date column found for {state_name}. Skipping this state.")
                continue
        
        df['state'] = state_name
        df['region'] = state_to_region[state_name]
        all_data.append(df)
    else:
        print(f"⚠️ No data for {state_name}")

# Combine all data
final_df = pd.concat(all_data, ignore_index=True)

# Save to Excel
final_df.to_excel("India_Region_Weather_2023_2025.xlsx", index=False)

print("✅ All data saved to 'India_Region_Weather_2023_2025.xlsx'")


Fetching weather data:   3%|█▊                                                          | 1/33 [00:00<00:23,  1.35it/s]

Columns for Delhi: Index(['date', 'tavg', 'tmin', 'tmax', 'prcp', 'wspd', 'pres', 'tsun'], dtype='object')
⚠️ No 'month' column for Delhi, checking available columns.


Fetching weather data:   6%|███▋                                                        | 2/33 [00:01<00:20,  1.51it/s]

Columns for Haryana: Index(['date', 'tavg', 'tmin', 'tmax', 'prcp', 'wspd', 'pres', 'tsun'], dtype='object')
⚠️ No 'month' column for Haryana, checking available columns.


Fetching weather data:   9%|█████▍                                                      | 3/33 [00:01<00:18,  1.62it/s]

...

Columns for Manipur: Index(['date', 'tavg', 'tmin', 'tmax', 'prcp', 'wspd', 'pres', 'tsun'], dtype='object')
⚠️ No 'month' column for Manipur, checking available columns.


Fetching weather data:  97%|█████████████████████████████████████████████████████████▏ | 32/33 [00:19<00:00,  1.73it/s]

Columns for Nagaland: Index(['date', 'tavg', 'tmin', 'tmax', 'prcp', 'wspd', 'pres', 'tsun'], dtype='object')
⚠️ No 'month' column for Nagaland, checking available columns.


Fetching weather data: 100%|███████████████████████████████████████████████████████████| 33/33 [00:19<00:00,  1.69it/s]

Columns for Mizoram: Index(['date', 'tavg', 'tmin', 'tmax', 'prcp', 'wspd', 'pres', 'tsun'], dtype='object')
⚠️ No 'month' column for Mizoram, checking available columns.
✅ All data saved to 'India_Region_Weather_2023_2025.xlsx'
